In [2]:
from skyfield.api import load, wgs84

stations_url = 'https://celestrak.org/NORAD/elements/gp.php?GROUP=starlink&FORMAT=tle'
satellites = load.tle_file(stations_url)
print('Loaded', len(satellites), 'satellites')
by_name = {sat.name: sat for sat in satellites}
satellite = by_name['STARLINK-1007']

# year, month, day, hour, minute, second
ts=load.timescale()
t = ts.now()
a=satellite.at(t)
lat, lon = wgs84.latlon_of(a)
print('Latitude:', lat)
print('Longitude:', lon)

Loaded 4219 satellites
Latitude: 51deg 51' 11.6"
Longitude: 24deg 40' 18.2"


In [3]:
from common import calculate_visible_satellites

# garching coordinates
observer_latitude = 48.2489
observer_longitude = 11.6532
observer_elevation = 0

visible_satellites = calculate_visible_satellites(
    observer_latitude, observer_longitude, observer_elevation, distance_km=800
)

for sat, alt, az in visible_satellites:
    print(sat.name)
    # print(f"Satellite: {sat}")
    # print(f"Altitude: {alt.degrees} degrees")
    # print(f"Azimuth: {az.degrees} degrees")

STARLINK-1048
STARLINK-1297
STARLINK-1286
STARLINK-1472
STARLINK-1512
STARLINK-1534
STARLINK-2445
STARLINK-2455
STARLINK-2658
STARLINK-2253
STARLINK-2688
STARLINK-3331
STARLINK-3791
STARLINK-3798
STARLINK-3933
STARLINK-4106
STARLINK-4258
STARLINK-4273
STARLINK-4056
STARLINK-4129
